# Data Augmentation & Transfer Learning

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from keras.preprocessing import image

In [ ]:
batch_size = 16
img_size = 150
train_path = '../data/catsdogs/train/'
test_path = '../data/catsdogs/test/'

## Data augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest')

In [ ]:
input_path = train_path + 'cats/cat.1.jpg'

img = image.load_img(input_path, target_size=(img_size, img_size))
img

In [ ]:
img_array = image.img_to_array(img)

In [ ]:
img_tensor = np.expand_dims(img_array, axis=0)

In [ ]:
plt.figure(figsize=(12, 12))

i = 0
for img in datagen.flow(img_tensor, batch_size=1):
    i += 1
    if i > 16:
        break
    plt.subplot(4, 4, i)
    plt.imshow(img[0])

## Exercise 1

`datagen` also has a method called `flow_from_directory`. Use it to produce a batch of 16 images of cats and dogs flowing from the training directory.

In [ ]:
train = datagen.flow_from_directory(train_path,
                                    target_size = (img_size, img_size),
                                    batch_size = batch_size,
                                    class_mode = 'binary')

In [ ]:
batch, labels = train.next()

In [ ]:
ld = {v: k for k, v in train.class_indices.items()}

In [ ]:
labels_s = [ld[i] for i in labels]

In [ ]:
plt.figure(figsize=(12, 12))
i = 1
for img, label in zip(batch, labels_s):
    plt.subplot(4, 4, i)
    plt.imshow(img)
    plt.title(label)
    i += 1

plt.tight_layout()

## Convolutional 

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_size, img_size, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
history = model.fit_generator(
                    train_generator,
                    steps_per_epoch=2000 // batch_size,
                    epochs=15,
                    validation_data=test_generator,
                    validation_steps=800 // batch_size)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Accuracy')
plt.legend(['train', 'test'])
plt.xlabel('Epochs')

## Bottleneck features

In [ ]:
from keras.applications import VGG16

In [ ]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(img_size, img_size, 3))

In [ ]:
base_model.summary()

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
def generate_bottlenecks(model,
                         input_path,
                         img_size,
                         data_size,
                         batch_size=16,
                         verbose=1):
    generator = datagen.flow_from_directory(
    train_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)
    
    return model.predict_generator(generator,
                                   data_size // batch_size,
                                   verbose=verbose)

In [ ]:
bf_train = generate_bottlenecks(base_model, train_path, img_size, 2000, batch_size)

In [ ]:
import os
os.makedirs('models', exist_ok = True)

In [ ]:
np.save(open('models/bf_train.npy', 'wb'),
        bf_train)

In [ ]:
bf_test = generate_bottlenecks(base_model, test_path, img_size, 800, batch_size)

In [ ]:
np.save(open('models/bf_test.npy', 'wb'),
        bf_train)

## Train a fully connected on bottlenecks

In [ ]:
X_train = np.load(open('models/bf_train.npy', 'rb'))
X_test = np.load(open('models/bf_test.npy', 'rb'))

In [ ]:
X_train.shape

In [ ]:
y_train = np.array([0] * 1000 + [1] * 1000)
y_test = np.array([0] * 400 + [1] * 400)

In [ ]:
fc_model = Sequential()

fc_model.add(Flatten(input_shape=train_data.shape[1:]))
fc_model.add(Dense(256, activation='relu'))
fc_model.add(Dropout(0.5))
fc_model.add(Dense(1, activation='sigmoid'))

fc_model.compile(optimizer='rmsprop',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

In [ ]:
history = fc_model.fit(X_train, y_train,
                       epochs=30,
                       batch_size=batch_size,
                       validation_data=(X_test, y_test))

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Accuracy')
plt.legend(['train', 'test'])
plt.xlabel('Epochs')

In [ ]:
fc_model.save_weights('models/top_model_weights.h5')

## Exercise 2

Try changing the value of Dropout probability and the number of nodes in the dense layer and see if you can get the model to overfit a little less.
Once you are satisfied remember to save the weights with the command above.

## Fine Tune full model

In [ ]:
fc_model.load_weights('models/top_model_weights.h5')

In [ ]:
full_model = Sequential()
for layer in base_model.layers: 
    full_model.add(layer)

for layer in fc_model.layers:
    full_model.add(layer)

In [ ]:
full_model.summary()

In [ ]:
for layer in full_model.layers[:15]:
    layer.trainable = False

In [ ]:
full_model.summary()

In [ ]:
from keras.optimizers import SGD

In [ ]:
full_model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [ ]:
full_model.evaluate_generator(test_generator, steps=50)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
full_model.fit_generator(
              train_generator,
              steps_per_epoch=125,
              epochs=5,
              validation_data=test_generator,
              validation_steps=50)

In [ ]:
full_model.evaluate_generator(test_generator, 50)

## Exercise 3

At https://keras.io/applications/#documentation-for-individual-models you can find all the pre-trained models available in keras.

- Try using another model, for example Xception and repeat the training. Do you get to a better accuracy?
- Can you rewrite this code using the functional API?

check out:
- https://keras.io/applications/#usage-examples-for-image-classification-models
- https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
- https://keras.io/applications/

*Copyright &copy; 2017 CATALIT LLC.  All rights reserved.*